In [1]:
from scipy.sparse import csr_matrix, lil_matrix, load_npz, save_npz, hstack
import numpy as np 
import pandas as pd 
from tqdm import tqdm 
import os

In [2]:
tuner_output_baseline_w_aux = '../aux_data_preperation/baseline_plus_aux_data/'
path = os.path.join(tuner_output_baseline_w_aux, 'matrices/cls/cls_T10_y.npz')
y_aux = load_npz(path)
y_aux

<422240x2981 sparse matrix of type '<class 'numpy.float32'>'
	with 4882954 stored elements in Compressed Sparse Row format>

In [3]:
y_sparse_main_fold0 = lil_matrix(y_aux.shape)

In [4]:
# Generate the y-sparse matrix for inference 
# Main tasks + fold 0 to limit the amount of predictions

In [5]:
path = os.path.join(tuner_output_baseline_w_aux, 'matrices/cls/cls_T11_fold_vector.npy')
folds = np.load(path)

In [6]:
path = './files/mapping/baseline_image_model_baselineaux_task_mapping.csv'
df_matching = pd.read_csv(path)

In [7]:
# main tasks 
# mind the difference between these two !
# cont_classification_task_id is the baseline + aux model 
display(df_matching.query('validity_0 >= 0').query('validity_1 >= 0')['cont_classification_task_id'].tail())
display(df_matching.query('validity_0 >= 0').query('validity_1 >= 0')['cont_classification_task_id_baseline'].tail())

50   NaN
51   NaN
52   NaN
53   NaN
54   NaN
Name: cont_classification_task_id, dtype: float64

50    1710.0
51    1898.0
52    2941.0
53    2942.0
54    2944.0
Name: cont_classification_task_id_baseline, dtype: float64

In [8]:
lst = [0
       ,0.2
       ,0.4
       ,0.5
       ,0.6
       ,0.7
       ,0.8
       ,0.9
       ,0.95
       ,0.99
      ]

In [9]:
for i,l1 in tqdm(enumerate(lst)):
    l2 = l1
    print(df_matching.query('validity_0 > @l1').query('validity_1 > @l2')['cont_classification_task_id'].nunique())

10it [00:00, 139.25it/s]

30
27
16
10
5
1
0
0
0
0


In [10]:
# new approach : for inference, only consider the main tasks that correspond to aux tasks meeting the criteria 
# instead of one general sparse y matrix 

base_path = '../aux_data_preperation/baseline_plus_aux_data/matrices/cls/fold_0'
os.makedirs(base_path, exist_ok=True)

for i,l1 in tqdm(enumerate(lst)):
    y_sparse_main_fold0 = lil_matrix(y_aux.shape)
    l2 = l1
    #for j,l2 in enumerate(lst): 
    file_core = 'ppv{}_npv{}'.format(l1,l2).replace('.','_')
    col_idxs = set(df_matching.query('validity_0 > @l1').query('validity_1 > @l2')['cont_classification_task_id'].dropna())
    for col in col_idxs :
        y_sparse_main_fold0[folds == 0,col] = 1
    path = os.path.join(base_path, 'y_sparse_main_tasks_fold0_{}'.format(file_core))
    save_npz(path,csr_matrix(y_sparse_main_fold0))

10it [00:06,  1.54it/s]
